# Installing packages section

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml==1.1.6

# Importing packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import joblib
#from fosforio import snowflake
from fosforml import *
from fosforml.constants import MLModelFlavours
from fosforio import get_dataframe
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import configparser
from dateutil.relativedelta import relativedelta
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

# Fetching the datasets

In [ ]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [ ]:
table_name = 'BOOKINGS_TRANSFORMED'

In [ ]:
from fosforio import snowflake
from fosforio import get_dataframe

snowflake.get_connection(connection_name="ME_AD_SALES_CXN")

In [ ]:
df_all = get_dataframe("DF_ALL")
df_opt = get_dataframe("DF_OPT")

In [ ]:
df_all

In [ ]:
df_all.info()

In [ ]:
df_opt.info()

In [ ]:
df_opt.columns = df_opt.columns.str.lower()
df_all.columns = df_all.columns.str.lower()

# Exploratory data analysis

In [ ]:
cat_col=['date','geo_id','order_id','site_id', 'ad_type_id','device_type', 'advertiser_id',
         'line_item_type', 'os_type','monetization_channel']
scat_col = ['site_id', 'ad_type','device_type', 'advertiser_id',
         'line_item_type', 'os_type','monetization_channel']
num_col=list(df_all.select_dtypes(np.number).columns)

In [ ]:
null_counts = df_all.isnull().sum()

null_counts = null_counts[null_counts > 0]

if not null_counts.empty:
    plt.figure(figsize=(12, 8))
    null_counts.plot(kind='bar', color='skyblue')
    plt.title('Count of Null Values in Each Column')
    plt.xlabel('Columns')
    plt.ylabel('Number of Null Values')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("There are no null values in the DataFrame.")

# bar plot

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(14, 12))

axes_ = [axes_row for axes in ax for axes_row in axes]

for i, col in enumerate(scat_col):
    if col in df_all.columns:
        sns.countplot(data=df_opt, x=col, ax=axes_[i])
        if col == 'advertiser_id':
            axes_[i].set_xticklabels(axes_[i].get_xticklabels(), rotation=90)
    else:
        axes_[i].text(0.5, 0.5, f'Column {col} not found', ha='center', va='center')
        axes_[i].set_axis_off()

plt.tight_layout()

plt.show()

# Scatter plot using total_revenue

In [ ]:
for i in num_col:
    if i!='total_revenue':
        sns.scatterplot(data=df_opt,x=i,y='total_revenue')
        plt.show()

In [ ]:
for col in (scat_col):
    title='Relationship of '+col+' with total_revenue'
    plt.figure(figsize=(6,4))
    sns.barplot(y=df_all['total_revenue'],x=df_all[col])
    if col=='advertiser_id':
        plt.xticks(rotation=90)
    plt.title(title)
    plt.show()

In [ ]:
for col in (scat_col):
    title='Relationship of '+ col +' with total_impressions'
    plt.figure(figsize=(6,4))
    sns.barplot(y=df_all['total_impressions'],x=df_all[col],)
    if col=='advertiser_id':
        plt.xticks(rotation=90)
    plt.title(title)
    plt.show()

In [ ]:
for i in df_all.select_dtypes(object).columns:
    pd.crosstab(df_all['monetization_channel'],df_all[i]).plot(kind='bar')
    plt.show()